# **SEMI-SUPERVISED LEARNING ON FASHION MNIST DATASET**

In [ ]:
!pip install ipython-autotime --quiet
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 12 s (started: 2024-04-21 14:18:26 +00:00)


#  Import Necessary Models

In [ ]:
# import necessary models
from tqdm import tqdm
import numpy as np
from tensorflow import keras
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.linear_model import LogisticRegression

time: 1.08 ms (started: 2024-04-21 14:18:57 +00:00)


# Load Fashion MNIST data

Fashion MNIST dataset can be easily accessed in Python through the Keras library, which provides a simple interface to download and load the dataset.

In [ ]:
# Load Fashion MNIST data

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step
time: 3.84 s (started: 2024-04-21 14:19:06 +00:00)


# Data Preprocessing

The following preprocessing steps standardize the pixel values and reshape the image data into a format suitable for feeding into a machine learning model, typically for tasks such as image classification.

In [ ]:
# Preprocess data

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

time: 179 ms (started: 2024-04-21 14:19:13 +00:00)


# A function to print a particular image

In [ ]:
# Printing a particular image

get_original_array = lambda arr: np.array([int(x * 255) for x in arr], dtype=np.uint8)

def print_image(i, data="train"):
  if type(i) is int:
    if data == 'train':
      return get_original_array(X_train[i]).reshape(28,28)
    else:
      return get_original_array(X_test[i]).reshape(28,28)
  else:
    return get_original_array(i).reshape(28,28)
#example
print_image(48)

array([[  0,   0,   0,   0,   0,   0,   0,   0,  39,  90, 135, 200, 132,
         60,  64,  83, 183, 195,  97,  29,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,  35, 125, 185, 200, 191, 159, 176, 233,
        237, 255, 240, 212, 164, 173, 201, 191, 128,  10,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  29, 172, 171, 159, 150, 145, 155, 132, 128,
        149, 152, 138, 129, 143, 151, 147, 156, 171, 169,  40,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 158, 164, 151, 152, 151, 151, 150, 143, 142,
        136, 135, 145, 150, 152, 150, 155, 155, 151, 164, 142,   0,   0,
          0,   0],
       [  0,   0,   0,  31, 164, 153, 161, 154, 150, 149, 151, 169, 172,
        171, 168, 175, 155, 144, 150, 149, 150, 156, 159, 175,  51,   0,
          0,   0],
       [  0,   0,   0,  95, 173, 162, 157, 153, 149, 149, 174, 115,  36,
         12,  23,  55, 146, 170, 150, 148, 145, 154, 166, 179, 106,   0,
          0,   0],
       [  0,   0,   0, 151, 170, 174, 173, 171, 151, 175,  93,  21, 108,
        144, 139,  69,   9, 124, 174, 143, 150, 162, 172, 180, 156,   0,
          0,   0],
       [  0,   0,   0, 179, 162, 180, 180, 148, 171, 126,   0, 129, 142,
        179, 167, 213, 133,   5, 147, 157, 138, 177, 185, 177, 179,   0,
          0,   0],
       [  0,   0,  22, 163, 167, 173, 198, 165, 190,  28,  34, 142, 181,
        177, 145, 140, 210,   0, 103, 181, 147, 208, 198, 165, 175,  44,
          0,   0],
       [  0,   0,  82, 176, 171, 175, 212, 178, 202,  37,  21, 116,  63,
         71,  68,  63,  94,   0, 129, 176, 168, 229, 191, 163, 179,  82,
          0,   0],
       [  0,   0,  60, 163, 174, 212, 186, 124, 210,  47,  20, 190, 147,
        152, 165, 212, 169,   0, 171, 176,  98, 208, 215, 170, 174, 104,
          0,   0],
       [  0,   0,   0,   0,   0,  49,  13, 101, 185, 157,  52, 150, 208,
        198, 193, 170,  63,  90, 183, 171, 112,   0,  45,   0,   0,   0,
          0,   0],
       [  0,   0,   1,   0,   0,   0,   0, 139, 155, 159, 138,  49,  21,
        118,  97,  55,  85, 169, 163, 158, 136,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   1,   4,   0, 111, 160, 147, 183, 166, 101,
        112, 100, 144, 180, 162, 158, 161, 130,   0,   0,   2,   2,   2,
          0,   0],
       [  0,   0,   0,   0,   0,   1,   0,  90, 153, 154, 155, 168, 186,
        185, 186, 174, 160, 158, 156, 162, 130,   0,   0,   0,   1,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   1,   0,  85, 156, 158, 159, 156, 156,
        158, 158, 155, 157, 162, 152, 165, 132,   0,   0,   1,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   1,   0,  78, 162, 159, 158, 162, 164,
        162, 164, 162, 158, 162, 155, 165, 148,   0,   0,   1,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   1,   0,  84, 164, 157, 159, 165, 165,
        161, 164, 163, 157, 160, 161, 162, 161,   0,   0,   1,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   1,   0,  95, 167, 155, 158, 161, 163,
        161, 168, 166, 158, 162, 163, 162, 158,   0,   0,   1,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   2,   0,  99, 169, 155, 160, 162, 164,
        163, 168, 163, 161, 162, 157, 163, 161,   0,   0,   2,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   2,   0,  99, 170, 154, 159, 160, 162,
        164, 167, 161, 162, 159, 154, 162, 169,   0,   0,   2,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   1,   0,  94, 167, 155, 160, 162, 164,
        161, 168, 164, 158, 159, 164, 158, 174,   0,   0,   2,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   2,   0,  96, 169, 156, 159, 162, 163,
        162, 168, 171, 163, 159, 163, 155, 176,   0,   0,   2,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   2,   0,  99, 167, 157, 165, 164, 163,
        165, 168, 169, 164, 161, 162, 155, 175,   0,   0,   2,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   2,   

time: 21 ms (started: 2024-04-21 15:20:15 +00:00)


# Standard Logistic Regression

At first, we fit a standard logistic regression model on the Fashion MNIST dataset and observe the accuracy score.

In [ ]:
# Standard logistic model has a very high accuracy for the dataset.

logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_train, y_train)
logistic_regressor_clf.score(X_test, y_test)

0.8442

time: 9min 57s (started: 2024-04-21 14:37:10 +00:00)


Note that the standard logistic regression model gives an accuracy score of 0.8442 for the Fashion MNIST dataset (which is a pretty high score).

# **Now, we will perform semi-supervised learning on the Fashion MNIST dataset and compare the accuracy scores.**

1.   First, we will take 100 random samples from the dataset use it to train the
 logistic regression model.
2.   Next, we will use k-means clustering to get 100 clusters and take the nearest points as centroids. Then we will use those 100 data-points to train the logistic regression classifier.
3. Then we will propagate the labelling to the whole cluster and do the same.
4. Repeat the same, but only propagate to the 20% of the dataset now.



# Perform Logistic Regression On Randomly Selected 100 Samples

In [ ]:
# Randomly select 100 samples from the training data
idx = np.random.choice(X_train.shape[0], 100, replace=False)

# Get the selected samples and their corresponding labels
X_sample = X_train[idx]
y_sample = y_train[idx]

logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_sample, y_sample)
logistic_regressor_clf.score(X_test, y_test)

0.6925

time: 662 ms (started: 2024-04-21 14:19:49 +00:00)


An accuracy score of 0.6925 is obtained when we use randomly selected 100 samples from the dataset to train the model. Note that the accuracy is lesser than the standard logistic regression model.

# Perform Logistic Regression On 100 Cluster Representative Points

In [ ]:
# Use KMeans to select 100 representative samples and then train the classifier

kmeans = MiniBatchKMeans(n_clusters=100, random_state=42)
kmeans.fit(X_train)

cluster_centers = kmeans.cluster_centers_

nearest_points_to_cluster = [-1]*100
distance_from_nearest_points_to_cluster = [10e9]*100

# Select those representative points
for i, cluster_center in enumerate(cluster_centers):
  for j, data_point in enumerate(X_train):
    distance = np.linalg.norm(cluster_center-data_point)
    if distance < distance_from_nearest_points_to_cluster[i]:
      distance_from_nearest_points_to_cluster[i] = distance
      nearest_points_to_cluster[i] = j

X_sample_ = X_train[nearest_points_to_cluster]
y_sample_ = y_train[nearest_points_to_cluster]

# Train and test the logistic regression classifier
logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_sample_, y_sample_)
logistic_regressor_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.7078

time: 49.2 s (started: 2024-04-21 14:19:54 +00:00)


 We observe that the accuracy  score for the above model has improved but is still less than the standard logistic regression model.

# Perform Logistic Regression On 100 Cluster Representative Points After Propagating the Labels

In [ ]:
# Use KMeans to select 100 representative samples, propagate the labels and then train the classifier

kmeans = MiniBatchKMeans(n_clusters=100, random_state=42)
kmeans.fit(X_train)

cluster_centers = kmeans.cluster_centers_
datapoints_labels = kmeans.labels_

# Nearest points of each cluster
nearest_points_to_cluster = [-1]*100
distance_from_nearest_points_to_cluster = [10e9]*100

# Mapping between labels and nearest points
nearest_point_of_a_label = [-1]*100

# Select those representative points
for i, cluster_center in enumerate(cluster_centers):
  for j, data_point in enumerate(X_train):
    distance = np.linalg.norm(cluster_center-data_point)
    if distance < distance_from_nearest_points_to_cluster[i]:
      distance_from_nearest_points_to_cluster[i] = distance
      nearest_points_to_cluster[i] = j
      nearest_point_of_a_label[datapoints_labels[j]] = j

y_propagated = [-1]*(X_train.shape[0])
for i, datapoint in enumerate(X_train):
  cluster_label = datapoints_labels[i]
  y_propagated[i] = y_train[nearest_point_of_a_label[cluster_label]]

# Train and test the logistic regression classifier
logistic_regressor_clf = LogisticRegression(random_state=69, max_iter=10000).fit(X_train, y_propagated)
logistic_regressor_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.6992

time: 9min 51s (started: 2024-04-21 14:20:43 +00:00)


# Perform Logistic Regression On 100 Cluster Representative Points After Propagating the Labels Partially

In [ ]:
# Use KMeans to select 100 representative samples, propagate the labels partially and then train the classifier

kmeans = MiniBatchKMeans(n_clusters=100, random_state=42)
kmeans.fit(X_train)

cluster_centers = kmeans.cluster_centers_
datapoints_labels = kmeans.labels_

# Nearest points of each cluster
nearest_points_to_cluster = [-1]*100
distance_from_nearest_points_to_cluster = [10e9]*100

# Mapping between labels and nearest points
nearest_point_of_a_label = [-1]*100

# Select those representative points
for i in tqdm(range(len(cluster_centers))):
  for j, data_point in enumerate(X_train):
    distance = np.linalg.norm(cluster_centers[i]-data_point)
    if distance < distance_from_nearest_points_to_cluster[i]:
      distance_from_nearest_points_to_cluster[i] = distance
      nearest_points_to_cluster[i] = j
      nearest_point_of_a_label[datapoints_labels[j]] = j

closest_points_from_centroid = np.array([[1]*120]*100)

for i in tqdm(range(len(nearest_points_to_cluster))):
  distance_point_pair = np.empty((0, 2))

  for j, data_point in enumerate(X_train):

    dist = np.linalg.norm(data_point-X_train[nearest_points_to_cluster[i]])
    to_append = np.array([dist, j])
    distance_point_pair = np.concatenate((distance_point_pair, [to_append]))

  sorted_indices = np.argsort(distance_point_pair[:, 0])
  distance_point_pair = (distance_point_pair[sorted_indices])[:120]

  closest_points_idx = distance_point_pair[:, 1]
  closest_points_from_centroid[i] = closest_points_idx

X_sample = []
y_propagated = []

for i, id in enumerate(closest_points_from_centroid):
  for j in id:
    X_sample.append(X_train[j])
    y_propagated.append(y_train[nearest_points_to_cluster[i]])

# Train and test the logistic regression classifier
logistic_regressor_clf = LogisticRegression(random_state=37, max_iter=10000).fit(X_sample, y_propagated)
logistic_regressor_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 100/100 [04:37<00:00,  2.77s/it]


0.7041

time: 6min 34s (started: 2024-04-21 14:30:35 +00:00)


## Report:

- NOTE THAT THE NUMBER OF CLUSTERS TO BE MADE WAS CHOSEN BASED ON THE SIZE OF THE DATASET.
- Standard logistic model has a very high accuracy for the dataset. (`0.8442`)
- Now, we choose randomly 100 datapoints (out of 60000) and use standard logistic regression to get (`0.6925`). The performance was poor.
- Then, instead of choosing randomly, we cluster the dataset into 100 clusters and used the nearest points to the centroids as the representative points to train the standard logistic model. The accuracy was `0.7078`. The accuracy is greatly improved as we are now using the best of its kind to train.
- Next, we propagate the labels to every cluster point to get the accuracy as `0.6992`. Performance drops as there might be spurious labelling.
- Now, instead of propagating to all the points, we propagate to the 20% data (`7041`). Surprisingly, the performance improves implying labelling small dataset accordingly (semi-supervised learning) performs greatly.